In [19]:
%load_ext autoreload
%autoreload 2
import traceback
from processingmm import batch_processing_win7, batch_processing
from processingmm.helpers import load_filenames_raw_data, add_path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from processingmm.batch_processing import find_processed_folders, find_all_folders, create_folders_df

In [26]:
import os
import shutil 

chunk_size = 2
max_nb = 1

# 1. Backup the data in the B: drive
base_directory = [r'F:\test_dev\test']
target = [r'F:\test_dev\backup']
base_temp_directory = r'F:\test_dev\test_temp'
calib_directory = r'F:\polarimetry\calibration'
directory_computation = r'F:\test_dev\temp'
folder_NAS = r'D:\DataRaw'

In [23]:
def batch_process_win7(base_directory: list, target: list, base_temp_directory: str, calib_directory: str, 
                       directory_computation: str):
    batch_processing_win7.backup_to_B_drive(base_directory, target)
    
    processed = []
    
    df = batch_processing.get_df_processing(base_directory)
    to_process = batch_processing.get_to_process(df, run_all = False)[0:chunk_size]
    _ = batch_processing.move_folders_temp(base_directory, base_temp_directory, to_process)
    
    while len(to_process) > 0:
        try:
            shutil.rmtree(directory_computation)
        except FileNotFoundError:
            pass
        except:
            traceback.print_exc()
        try:
            os.mkdir(directory_computation)
        except FileExistsError:
            pass
        except:
            traceback.print_exc()

        to_process = batch_processing.batch_process([base_temp_directory], calib_directory, run_all = False, parameter_set = 'CUSA', 
                                                    max_nb = 1, target_temp = directory_computation)
            
        _ = batch_processing.move_folders_temp(directory_computation, base_temp_directory, to_process, put_back = True)

        try:
            shutil.rmtree(directory_computation)
        except:
            traceback.print_exc()

        df = batch_processing.get_df_processing([base_temp_directory])
        to_process = batch_processing.get_to_process(df, run_all = False)

    processed = batch_processing_win7.move_back_the_folders(base_directory, base_temp_directory)
    
    return processed

In [24]:
processed = batch_process_win7(base_directory, target, base_temp_directory, calib_directory, directory_computation)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:23<00:00, 23.94s/it]


In [35]:
def move_to_NAS(directories: list, processed: list, folder_NAS: str):
    
    already_done = set(os.listdir(folder_NAS))
    for folder in os.listdir(directories[0]):
        if folder in already_done:
            pass
        else:
            shutil.copytree(os.path.join(directories[0], folder), os.path.join(folder_NAS, folder))

In [36]:
# 5. Move the data to the NAS
move_to_NAS(base_directory, processed, folder_NAS)

In [ ]:
finished = True
while finished:
    batch_process_win7(base_directory, target, base_temp_directory, calib_directory, directory_computation)
    gc.collect()
    time.sleep(60)